In [ ]:
"""
This script converts a CSV file containing GPS data to a GPX file.

Inputs:
  - csv_file: Path to the input CSV file.
  - gpx_file: Path to the output GPX file.
  - utc_to_pst: Timezone offset from UTC to PST (in hours).
  - start_time: Start time for filtering data (in HH:MM:SS format).
  - end_time: End time for filtering data (in HH:MM:SS format).
  - bus_number: Bus number to filter data for.
  - reverse_timestamps: Flag to indicate whether to reverse the timestamps 
    (True for reverse, False for original order).
"""




from typing_extensions import dataclass_transform
import csv
import xml.etree.ElementTree as ET
from xml.dom import minidom
from datetime import datetime
from datetime import timedelta
import operator  # Import the operator module
from datetime import time




# Define the CSV input file and GPX output file
csv_file = '/content/2024-10-08.csv'
gpx_file = '/content/WC-2024-08_rev.gpx'
utc_to_pst = -7


def parse_time(time_string):
    # Split the time string into HH:MM and decimal part
    hours, minutes = time_string.split(':')
    minutes, decimal = minutes.split('.')

    # Convert hours, minutes, and fractional part
    hours = int(hours)
    minutes = int(minutes)
    fractional_minutes = float('0.' + decimal)  # convert decimal part to fractional minutes
    total_minutes = minutes + fractional_minutes
    seconds = fractional_minutes * 60

    # Convert total minutes to seconds and create a timedelta
    total_seconds = total_minutes * 60
    return hours, minutes, seconds

# Function to convert DMS to decimal degrees
def dms_to_decimal(degrees, minutes, seconds, direction):
    decimal = abs(float(degrees)) + float(minutes) / 60 + float(seconds) / 3600
    if direction in ['S', 'W']:
        decimal *= -1
    return decimal

# Define GPX XML structure
def create_gpx_root():
    gpx = ET.Element('gpx', version='1.1', creator='PythonScript', xmlns='http://www.topografix.com/GPX/1/1')
    return gpx

def knot2ms(speed_knot):

    try:
      speed_ms = float(speed_knot) * 0.514444
    except:
      speed_ms=0.0
    return speed_ms

def split_timestamp(timestamp):
    """
    Splits a timestamp string into hour, minute, and second components.

    Parameters:
    timestamp (str): A timestamp string in the format "YYYY-MM-DD HH:MM:SS.sss".

    Returns:
    tuple: A tuple containing (hour, minute, second) where each is an integer.
    """

    # Define the format of the timestamp
    timestamp_format = "%Y-%m-%d %H:%M:%S.%f"


    # Parse the timestamp string into a datetime object
    dt = datetime.strptime(timestamp, timestamp_format)

    # Extract hour, minute, and second
    hour = dt.hour
    minute = dt.minute
    second = dt.second

    return hour, minute, second

def offset_timestamp(timestamp_str, offset_hours):

    timestamp_format = "%Y-%m-%d %H:%M:%S.%f"


    # Parse the input timestamp string into a datetime object
    time_obj = datetime.strptime(timestamp_str, timestamp_format)

    # Create a timedelta object representing the timezone offset (in hours)
    offset = timedelta(hours=offset_hours)

    # Apply the offset to the datetime object
    new_time_obj = time_obj + offset

    # Convert the new datetime object back to a string in the same format
    new_timestamp_str = new_time_obj.strftime(timestamp_format)[:-3]  # Truncate to milliseconds

    return new_timestamp_str


def create_track_segment(parent, timestamp, lat, lon, speed, heading):
    trkpt = ET.SubElement(parent, 'trkpt', lat=str(lat), lon=str(lon))
    ET.SubElement(trkpt, 'time').text = timestamp
    ET.SubElement(trkpt, 'speed').text = speed
    ET.SubElement(trkpt, 'heading').text = heading

def main():
    # Create the root of the GPX document
    gpx = create_gpx_root()

    # Create a track and a segment under the root
    trk = ET.SubElement(gpx, 'trk')
    trkseg = ET.SubElement(trk, 'trkseg')

    i = 0
    data_array = []

    start_time = time(11, 32, 0)
    end_time = time(12, 2,0)
    reverse_timestamps = True

    # Open the CSV file and read data
    with open(csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader, None)  # Skip the header row
     
        for row in reader:
              data_array.append(row)
        
    data_array.reverse()
       
    for row in data_array:

        i=i+1

        if not row or len(row) < 14:
            continue


        hour, minute, second = split_timestamp(offset_timestamp(row[1],utc_to_pst)) #parse_time()
        current_time = time(hour, minute, second)
        #print(start_time, current_time, end_time, start_time <= current_time <= end_time, row[0],row[0]=='WA181500061522')
        if start_time <= current_time <= end_time and row[0] == 'WA181500061522':


          print(f"{hour}:{minute}:{second}")
          # Extract relevant fields
          print(f"found a location to add to GPX file {i}")

          timestamp = offset_timestamp(row[1],utc_to_pst)
          lat_deg = row[2]  # Latitude degrees
          lat_min = row[3]  # Latitude minutes
          lat_sec = row[4]  # Latitude seconds
          lon_deg = row[5]  # Longitude degrees
          lon_min = row[6]  # Longitude minutes
          lon_sec = row[7]  # Longitude seconds
          speed_gps = row[8]  # Speed
          heading = row[9] # compass bearing

          # Convert DMS to decimal degrees
          lat = dms_to_decimal(lat_deg, lat_min, lat_sec, 'N')  # Assuming 'N' for North
          lon = dms_to_decimal(lon_deg, lon_min, lon_sec, 'W')  # Assuming 'W' for West
          #speed = knot2ms(speed_gps)

          # Create track points
          create_track_segment(trkseg, timestamp, lat, lon, speed_gps, heading)



    # Convert the XML tree to a string with pretty formatting
    xml_str = ET.tostring(gpx, encoding='utf-8')
    parsed_str = minidom.parseString(xml_str)
    pretty_str = parsed_str.toprettyxml(indent="  ")

    # Write the pretty-formatted XML to the GPX file
    with open(gpx_file, 'w') as f:
        f.write(pretty_str)

    print(f"GPX file written to {gpx_file}")

    # Write GPX file
    #tree = ET.ElementTree(gpx)
    #tree.write(gpx_file, encoding='utf-8', xml_declaration=True)

if __name__ == "__main__":
    main()
